# Ojetivo deste código
* Monitoramento temporal dos canais da NDAQ

# Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import numpy.matlib 
import matplotlib.pyplot as plt
import glob
import time
import timeit
%matplotlib inline
from datetime import datetime

# Váriaveis padrões

In [2]:
STS          = ['FlashADC_1','FlashADC_2','FlashADC_3','FlashADC_4','FlashADC_5','FlashADC_6','FlashADC_7','FlashADC_8']
ST           = ['Slot_Number_6','Slot_Number_10','Slot_Number_12','Slot_Number_14','Slot_Number_16']
Num_NDAQ     = [6,10,12,14,16]

# Funções 

In [3]:
def contron_pac(mes):


    dia       = []
    cont_MES  = []
    cont0     = []
    diretorio = ["/data2/raw/*","/data3/raw/*"]
    ref1      = ['/data2/raw/AngraRun_T','/data3/raw/AngraRun_T']

    for j in range(len(diretorio)):
        List = glob.glob(diretorio[j]) # listando todos os dados
        for i in range(len(List)):
            if List[i][0:21] != ref1[j]:
                cont0.append(List[i])
        for i in range(len(cont0)):
            ts = int(np.array(cont0)[i][20:-8])
            datas  = (datetime.fromtimestamp(ts).strftime('%Y-%m-%d'))
            if datas[0:4] == '2019':
                if datas[5:7] == mes:
                    cont_MES.append(cont0[i])
                    if datas[8] == '0':
                        dia.append(datas[9])
                    if datas[8] == '1':
                        dia.append(datas[8:10])
                    if datas[8] == '2':
                        dia.append(datas[8:10])
                    if datas[8] == '3':
                        dia.append(datas[8:10])
      
    return cont_MES,dia

In [4]:
def separate_ndaqs(ind1,ind2):
    ndaq6 = np.where((ind1.values.T[0] ==6))[0]
    ndaq10 = np.where((ind1.values.T[0] ==10))[0]
    ndaq12 = np.where((ind1.values.T[0] ==12))[0]
    ndaq14 = np.where((ind1.values.T[0] ==14))[0]
    return ndaq6[:-1], ndaq10[:-1], ndaq12[:-1], ndaq14[:-1]

def align_data(ndaq6, ndaq10, ndaq12, ndaq14,time):
    i = 0
    j = 0
    k = 0
    l = 0
    aligned6 = []
    aligned10 = []
    aligned12 = []
    aligned14 = []
    
    slot6  = time['Timestamp'][ndaq6][1:]
    slot10 = time['Timestamp'][ndaq10][1:]
    slot12 = time['Timestamp'][ndaq12][1:]
    slot14 = time['Timestamp'][ndaq14][1:]
    
    for i in range(0,len(slot6)):
        value = slot6[slot6.index[i]]
        max_distortion = max(5,150*value/1000000)
        no_j = 0
        no_k = 0
        no_l = 0
        range_0 = min(20,len(slot10)-j)
        for m in range(j,j+range_0):
            if (abs(value - slot10[slot10.index[m]]))<max_distortion:
                j=m+1
                break
            if (m==j+range_0-1):
                no_j = 1
                break
        if range_0==0:
            no_j=1
        range_1 = min(20,len(slot12)-k)
        for m in range(k,k+range_1):
            if (abs(value - slot12[slot12.index[m]]))<max_distortion:
                k=m+1
                break
            if (m==k+range_1-1):
                no_k = 1
                break
        if range_1==0:
            no_k=1        
        range_2 = min(20,len(slot14)-l)
        for m in range(l,l+range_2):
            if (abs(value - slot14[slot14.index[m]]))<max_distortion:
                l=m+1
                break
            if (m==l+range_2-1):
                no_l = 1
                break
        if range_2==0:
            no_l=1
        
        si = slot6.index[i]
        sj = slot10.index[j-1]
        sk = slot12.index[k-1]
        sl = slot14.index[l-1]
        if (no_j == 0) and (no_k == 0) and (no_l == 0):
            aligned6.append(si)
            aligned10.append(sj)
            aligned12.append(sk)
            aligned14.append(sl)
    return aligned6, aligned10, aligned12, aligned14

In [9]:
def cont_PMT(pac,STS,thr):
    ind1 = pd.read_parquet(pac, columns=['Slot Number'])

    ind2 = pd.read_parquet(pac, columns=['BusyFlag'])
    time = pd.read_parquet(pac,columns=['Timestamp'])
    ndaq6, ndaq10, ndaq12, ndaq14=separate_ndaqs(ind1,ind2)
    
    #aligned6, aligned10, aligned12, aligned14 = align_data(ndaq6, ndaq10, ndaq12, ndaq14,time) 
   
    aligned6 = ndaq6
    aligned10 = ndaq10
    aligned12 = ndaq12
    aligned14 = ndaq14

    CNDAQ6  = np.zeros((len(aligned6),8))
    CNDAQ10 = np.zeros((len(aligned10),8))
    CNDAQ12 = np.zeros((len(aligned12),8))
    CNDAQ14 = np.zeros((len(aligned14),8))

    ANDAQ6  = np.zeros((len(aligned6),8))
    ANDAQ10 = np.zeros((len(aligned10),8))
    ANDAQ12 = np.zeros((len(aligned12),8))
    ANDAQ14 = np.zeros((len(aligned14),8))

    PNDAQ6  = np.zeros((len(aligned6),8))
    PNDAQ10 = np.zeros((len(aligned10),8))
    PNDAQ12 = np.zeros((len(aligned12),8))
    PNDAQ14 = np.zeros((len(aligned14),8))


    mask = np.array([1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])

    for i in range(len(STS)):
        df         = pd.read_parquet(pac, columns=[STS[i]])
        
        S6         = np.stack(df.loc[aligned6]['FlashADC_'+str(1+i)])
        S10        = np.stack(df.loc[aligned10]['FlashADC_'+str(1+i)])
        S12        = np.stack(df.loc[aligned12]['FlashADC_'+str(1+i)])
        S14        = np.stack(df.loc[aligned14]['FlashADC_'+str(1+i)])
        
        del df
        #timestamp1 = np.array(time.loc[aligned6]['Timestamp'])
        
        baseline6  = np.dot(S6,mask)/mask.sum()
        baseline10 = np.dot(S10,mask)/mask.sum()
        baseline12 = np.dot(S12,mask)/mask.sum()
        baseline14 = np.dot(S14,mask)/mask.sum()
       
        Amp6  = baseline6
        Amp10 = baseline10
        Amp12 = baseline12
        Amp14 = baseline14
        
        sat6  = np.argmax(S6,  axis =1)
        sat10 = np.argmax(S10, axis =1)
        sat12 = np.argmax(S12, axis =1)
        sat14 = np.argmax(S14, axis =1)

        #Carga6  = np.sum(S6,   axis =1)  - baseline6*S6.shape[1]
        #Carga10 = np.sum(S10,  axis =1)  - baseline10*S10.shape[1]
        #Carga12 = np.sum(S12,  axis =1)  - baseline12*S12.shape[1]
        #Carga14 = np.sum(S14,  axis =1)  - baseline14*S14.shape[1]
        
        
        
        Carga6  = np.max(S6,   axis =1)  - baseline6
        Carga10 = np.max(S10,  axis =1)  - baseline10
        Carga12 = np.max(S12,  axis =1)  - baseline12
        Carga14 = np.max(S14,  axis =1)  - baseline14
        
        
      
        Amp6[Carga6<0]   = 0
        Amp10[Carga10<0] = 0
        Amp12[Carga12<0] = 0
        Amp14[Carga14<0] = 0

        Carga6[Carga6<0]   = 0
        Carga10[Carga10<0] = 0
        Carga12[Carga12<0] = 0
        Carga14[Carga14<0] = 0

        CNDAQ6[:,0 +i]  = Carga6
        CNDAQ10[:,0 +i] = Carga10
        CNDAQ12[:,0+i]  = Carga12
        CNDAQ14[:,0+i]  = Carga14

        ANDAQ6[:,0 +i]  = Amp6
        ANDAQ10[:,0 +i] = Amp10
        ANDAQ12[:,0+i]  = Amp12
        ANDAQ14[:,0+i]  = Amp14

        PNDAQ6[:,0 +i]  = sat6
        PNDAQ10[:,0 +i] = sat10
        PNDAQ12[:,0+i]  = sat12
        PNDAQ14[:,0+i]  = sat14
        
        Cmatrix      = np.concatenate((CNDAQ6,CNDAQ10,CNDAQ12,CNDAQ14), axis =1)
        Amatrix      = np.concatenate((ANDAQ6,ANDAQ10,ANDAQ12,ANDAQ14), axis =1)
        Pmatrix      = np.concatenate((PNDAQ6,PNDAQ10,PNDAQ12,PNDAQ14), axis =1)
    
    return Cmatrix,Amatrix,Pmatrix

def data_flame_dados(pac0):
    Cmatrix,Amatrix,Pmatrix = cont_PMT(pac0,STS,8)
    
    matrix = np.concatenate((Cmatrix.T,Amatrix.T,Pmatrix.T), axis = 0).T
    col=['PMT01','PMT02','PMT03', 'PMT04','PMT05','PMT06','PMT07','PMT08','PMT09','PMT10',
         'PMT11','PMT12','PMT13', 'PMT14','PMT15','PMT16','PMT17','PMT18','PMT19','PMT20',
         'PMT21','PMT22','PMT23', 'PMT24','PMT25','PMT26','PMT27','PMT28','PMT29','PMT30',
         'PMT31','PMT32','PMT01_p','PMT02_p','PMT03_p', 'PMT04_p','PMT05_p','PMT06_p','PMT07_p','PMT08_p','PMT09_p','PMT10_p',
         'PMT11_p','PMT12_p','PMT13_p','PMT14_p','PMT15_p','PMT16_p','PMT17_p','PMT18_p','PMT19_p','PMT20_p',
         'PMT21_p','PMT22_p','PMT23_p','PMT24_p','PMT25_p','PMT26_p','PMT27_p','PMT28_p','PMT29_p','PMT30_p',
         'PMT31_p','PMT32_p','PMT01_m','PMT02_m','PMT03_m','PMT04_m','PMT05_m','PMT06_m','PMT07_m','PMT08_m','PMT09_m','PMT10_m',
         'PMT11_m','PMT12_m','PMT13_m','PMT14_m','PMT15_m','PMT16_m','PMT17_m','PMT18_m','PMT19_m','PMT20_m',
         'PMT21_m','PMT22_m','PMT23_m', 'PMT24_m','PMT25_m','PMT26_m','PMT27_m','PMT28_m','PMT29_m','PMT30_m',
         'PMT31_m','PMT32_m']
    df = pd.DataFrame(matrix,columns=col, dtype='int32')
    return df

In [10]:
def media_std(df,df1,df2,df3,df4,col):
    a = df[col]
    b = df1[col]
    c = df2[col]
    d = df3[col]
    e = df4[col]
    #f = df5[col]
    
    matrix = np.concatenate((a,b,c,d,e))
    media  = np.mean(matrix, axis = 0)
    STD    =  np.std(matrix, axis = 0)
    
    return media, STD

In [11]:
def dados (pac):
    df  = data_flame_dados(pac[0])
    df1 = data_flame_dados(pac[1])
    df2 = data_flame_dados(pac[2])
    df3 = data_flame_dados(pac[3])
    df4 = data_flame_dados(pac[4])
    #df5 = data_flame_dados(pac[5])

    carga,    c_std = media_std(df,df1,df2,df3,df4,col_a)
    pedestal, p_std = media_std(df,df1,df2,df3,df4,col_b)
    amostra,  a_std = media_std(df,df1,df2,df3,df4,col_c)
    return carga, c_std ,pedestal, p_std,amostra,a_std

# Importando os arquivos no mês que eu quero

In [12]:
mes = '01'
cont_MES,dia = contron_pac(mes) # pacotes que estão neste mês e os dias que o pacote representa

In [13]:
dia  = np.array(dia)
cont_MES = np.array(cont_MES)

In [14]:
ponteiro_dia = np.sort(np.int64(np.unique(dia)))
ind_d        = np.arange(0,len(np.unique(dia)),3)

In [ ]:
for i in range(len(ind_d)):
    pac = np.array(cont_MES)[dia == str(ponteiro_dia[ind_d][i])]
    pac = np.unique(pac)
    #ind = np.arange(0,len(pac),round(len(pac)/6))
    ind = np.arange(6)
    #pac0 = pac[ind]
    if len(pac) < 40:
        relatorio[i] = 1 
        print('a')
    else:
        for j in range(len(ind)):
            for k in range(5):
                pac0 = pac[ind[j] + k]
                df         = pd.read_parquet(pac[ind[j]], columns=[STS[0]])
                if df.shape[0] == 0:
                    print(b)
                    relatorio[i,j] = 1
                    break

In [ ]:
if sum(np.sum(relatorio,axis = 1) > 0) >0:
    ind_delet = np.where(np.sum(relatorio,axis = 1) > 0)[0][0]
    ind_d = np.delete(ind_d,ind_delet)

In [15]:
col_a = ['PMT01','PMT02','PMT03', 'PMT04','PMT05','PMT06','PMT07','PMT08','PMT09','PMT10',
         'PMT11','PMT12','PMT13', 'PMT14','PMT15','PMT16','PMT17','PMT18','PMT19','PMT20',
         'PMT21','PMT22','PMT23', 'PMT24','PMT25','PMT26','PMT27','PMT28','PMT29','PMT30',
         'PMT31','PMT32']

col_as = ['PMT01s','PMT02s','PMT03s','PMT04s','PMT05s','PMT06s','PMT07s','PMT08s','PMT09s','PMT10s',
          'PMT11s','PMT12s','PMT13s','PMT14s','PMT15s','PMT16s','PMT17s','PMT18s','PMT19s','PMT20s',
          'PMT21s','PMT22s','PMT23s','PMT24s','PMT25s','PMT26s','PMT27s','PMT28s','PMT29s','PMT30s',
          'PMT31s','PMT32s']

col_b = ['PMT01_p','PMT02_p','PMT03_p', 'PMT04_p','PMT05_p','PMT06_p','PMT07_p','PMT08_p','PMT09_p','PMT10_p',
         'PMT11_p','PMT12_p','PMT13_p','PMT14_p','PMT15_p','PMT16_p','PMT17_p','PMT18_p','PMT19_p','PMT20_p',
         'PMT21_p','PMT22_p','PMT23_p','PMT24_p','PMT25_p','PMT26_p','PMT27_p','PMT28_p','PMT29_p','PMT30_p',
         'PMT31_p','PMT32_p']

col_bs = ['PMT01_ps','PMT02_ps','PMT03_ps','PMT04_ps','PMT05_ps','PMT06_ps','PMT07_ps','PMT08_ps','PMT09_ps','PMT10_ps',
          'PMT11_ps','PMT12_ps','PMT13_ps','PMT14_ps','PMT15_ps','PMT16_ps','PMT17_ps','PMT18_ps','PMT19_ps','PMT20_ps',
          'PMT21_ps','PMT22_ps','PMT23_ps','PMT24_ps','PMT25_ps','PMT26_ps','PMT27_ps','PMT28_ps','PMT29_ps','PMT30_ps',
          'PMT31_ps','PMT32_ps']

col_c = ['PMT01_m','PMT02_m','PMT03_m','PMT04_m','PMT05_m','PMT06_m','PMT07_m','PMT08_m','PMT09_m','PMT10_m',
         'PMT11_m','PMT12_m','PMT13_m','PMT14_m','PMT15_m','PMT16_m','PMT17_m','PMT18_m','PMT19_m','PMT20_m',
         'PMT21_m','PMT22_m','PMT23_m','PMT24_m','PMT25_m','PMT26_m','PMT27_m','PMT28_m','PMT29_m','PMT30_m',
         'PMT31_m','PMT32_m']

col_cs = ['PMT01_ms','PMT02_ms','PMT03_ms','PMT04_ms','PMT05_ms','PMT06_ms','PMT07_ms','PMT08_ms','PMT09_ms','PMT10_ms',
         'PMT11_ms','PMT12_ms','PMT13_ms','PMT14_ms','PMT15_ms','PMT16_ms','PMT17_ms','PMT18_ms','PMT19_ms','PMT20_ms',
         'PMT21_ms','PMT22_ms','PMT23_ms','PMT24_ms','PMT25_ms','PMT26_ms','PMT27_ms','PMT28_ms','PMT29_ms','PMT30_ms',
         'PMT31_ms','PMT32_ms']

In [16]:
coluna = np.concatenate((col_a,col_as,col_b,col_bs,col_c,col_cs))

In [17]:
DUQm = []
DUQstd =[]
PEDm = []
PEDs =[]
AMOm =[]
AMOs = []
DIA  = []

for i in range(len(ind_d)):
    pac = np.array(cont_MES)[dia == str(ponteiro_dia[ind_d][i])]
    pac = np.unique(pac)
    ind = np.arange(6)
    #ind = np.arange(0,len(pac),round(len(pac)/6))
    pac = pac[ind]
    
    print(i)

    carga, c_std ,pedestal, p_std,amostra,a_std = dados(pac)
    DUQm.append(carga)
    DUQstd.append(c_std)

    PEDm.append(pedestal)
    PEDs.append(p_std)

    AMOm.append(amostra)
    AMOs.append(a_std)

    DIA.append(ponteiro_dia[ind_d][i])

0
1
2
3
4
5
6
7
8
9
10


In [ ]:
DUQm   = np.stack(DUQm)
DUQstd = np.stack(DUQstd)

AMOm = np.stack(AMOm)
AMOs = np.stack(AMOm)

PEDm = np.stack(PEDm)
PEDs = np.stack(PEDs)

DIA = np.stack(DIA)

In [ ]:
D = np.arange(1,32) # Dias do mês de abril

In [ ]:
NDAQ_C  = np.zeros((len(D),32))
NDAQ_Cs = np.zeros((len(D),32))
NDAQ_P  = np.zeros((len(D),32))
NDAQ_Ps = np.zeros((len(D),32))
NDAQ_A  = np.zeros((len(D),32))
NDAQ_As = np.zeros((len(D),32))

In [ ]:
NDAQ_C[DIA-1]  = DUQm
NDAQ_Cs[DIA-1] = DUQstd

NDAQ_P[DIA-1]  = PEDm
NDAQ_Ps[DIA-1] = PEDs

NDAQ_A[DIA-1]  = AMOm
NDAQ_As[DIA-1] = AMOs

In [ ]:
NDAQ_C[NDAQ_C == 0] = np.NAN
NDAQ_Cs[NDAQ_Cs == 0] = np.NAN

NDAQ_P[NDAQ_P == 0] = np.NAN
NDAQ_Ps[NDAQ_Ps == 0] = np.NAN

NDAQ_A[NDAQ_A == 0] = np.NAN
NDAQ_As[NDAQ_As == 0] = np.NAN

In [ ]:
matrix = np.concatenate((NDAQ_C,NDAQ_Cs,NDAQ_P,NDAQ_Ps,NDAQ_A,NDAQ_As), axis = 1)

In [ ]:
df = pd.DataFrame(matrix,columns=coluna)

In [ ]:
df.to_parquet('Jan_va.parq')